# Customer Relationship Management (CRM): Recency Frequency Monetary (RFM) Analysis

CRM is Customer Relationship Management. Of course, there is also analytics. The essence is to get more revenue, get to know customers and make them feel special. CRM Analytics is an analysis that includes topics such as analyzing customer data (purchase information, etc.), getting to know customers better, segmenting customers and making decisions based on these segments (special offers, etc.). In the most basic terms, the aim is to recognize customer behaviors based on past data and to determine strategies according to these behaviors.

# 1. Importing Libraries and Dataset

* Business Problem: 
    * An e-commerce company wants to segment its customers and determine marketing strategies according to these segments.
* Data Set Story
    * https://archive.ics.uci.edu/ml/datasets/Online+Retail+II
    * The dataset named Online Retail II contains the sales of a UK-based online retail store between 01/12/2009 - 09/12/2011.
* Variables:
    * InvoiceNo: Invoice number. Unique number for each transaction, i.e. invoice. If it starts with C, canceled transaction.
    * StockCode: Product code. Unique number for each product.
    * Description: Product name
    * Quantity: Product quantity. It expresses how many of the products in the invoices are sold.
    * InvoiceDate: Invoice date and time.
    * UnitPrice: Product price (in pounds sterling)
    * CustomerID: Unique customer number
    * Country: Country name. The country where the customer lives.

We then use pd.set_option() to set the display options for a DataFrame. The first option, "display.max_column", sets the maximum number of columns that will be displayed in the output to None, meaning all columns will be displayed. The second option, "display.max_rows", sets the maximum number of rows that will be displayed in the output to None, meaning all rows will be displayed.

Finally, we use pd.set_option() again to set the float formatting for the output to show three decimal places. The lambda x: "%.3f" % x function formats the float to three decimal places using the string formatting syntax.

In [31]:
# Importing Libraries
import datetime as dt
import pandas as pd

# Set display options to show all columns and rows of a DataFrame
pd.set_option("display.max_column", None)
pd.set_option("display.max_rows", None)

# Set the float formatting to show three decimal places
pd.set_option("display.float_format", lambda x: "%.3f" % x)

In [32]:
# Read the Excel file into a DataFrame
df_ = pd.read_excel(r"/kaggle/input/online-retail-dataset/online_retail_II.xlsx", sheet_name="Year 2009-2010")
df = df_.copy()

df.head(10)

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.950,13085.000,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.750,13085.000,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.750,13085.000,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.100,13085.000,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.250,13085.000,United Kingdom
5,489434,22064,PINK DOUGHNUT TRINKET POT,24,2009-12-01 07:45:00,1.650,13085.000,United Kingdom
6,489434,21871,SAVE THE PLANET MUG,24,2009-12-01 07:45:00,1.250,13085.000,United Kingdom
7,489434,21523,FANCY FONT HOME SWEET HOME DOORMAT,10,2009-12-01 07:45:00,5.950,13085.000,United Kingdom
8,489435,22350,CAT BOWL,12,2009-12-01 07:46:00,2.550,13085.000,United Kingdom
9,489435,22349,"DOG BOWL , CHASING BALL DESIGN",12,2009-12-01 07:46:00,3.750,13085.000,United Kingdom


# 2. Exploratory Data Analysis

If the customer id is not clear, it disrupts customer segmentation, we will remove them

In [33]:
# Missing Values
print(df.isnull().sum())
print("---------------")

print("Number of lines before removing missing values: ", df.shape[0])

# Dropping Missing Values
df.dropna(inplace=True)

print("Number of lines after removing missing values: ", df.shape[0])

Invoice             0
StockCode           0
Description      2928
Quantity            0
InvoiceDate         0
Price               0
Customer ID    107927
Country             0
dtype: int64
---------------
Number of lines before removing missing values:  525461
Number of lines after removing missing values:  417534


In [34]:
#Dataframe does not have a total cost expression, to add it (per products)
df["TotalPrice"] = df["Quantity"] * df["Price"]

In [35]:
# Unique Invoice Count
print("Unique Invoice Count: ", df["Invoice"].nunique())

# Unique Customer Count
print("Unique Customer Count: ", df["Customer ID"].nunique())

Unique Invoice Count:  23587
Unique Customer Count:  4383


In [36]:
# How Many of the Products were Sold
df.groupby("Description").agg({"Quantity": "sum"}).sort_values("Quantity", ascending=False).head(10)

,Quantity
Description,
WHITE HANGING HEART T-LIGHT HOLDER,55861
WORLD WAR 2 GLIDERS ASSTD DESIGNS,54274
BROCADE RING PURSE,47430
PACK OF 72 RETRO SPOT CAKE CASES,44507
ASSORTED COLOUR BIRD ORNAMENT,44120
60 TEATIME FAIRY CAKE CASES,35630
PACK OF 60 PINK PAISLEY CAKE CASES,30888
JUMBO BAG RED RETROSPOT,29498
BLACK AND WHITE PAISLEY FLOWER MUG,25679


In [37]:
# Total Price per Invoice
df.groupby("Invoice").agg({"TotalPrice": "sum"}).head(10)

,TotalPrice
Invoice,
489434,505.300
489435,145.800
489436,630.330
489437,310.750
489438,2286.240
489439,426.300
489440,50.400
489441,344.340
489442,382.370


There are negative values, these indicate a refund. We need to get rid of them.

In [38]:
# Describe of Data
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Quantity,417534.000,12.759,101.220,-9360.000,2.000,4.000,12.000,19152.000
Price,417534.000,3.888,71.132,0.000,1.250,1.950,3.750,25111.090
Customer ID,417534.000,15360.645,1680.811,12346.000,13983.000,15311.000,16799.000,18287.000
TotalPrice,417534.000,19.994,99.916,-25111.090,4.250,11.250,19.350,15818.400


In [39]:
# Invoice starting with C means refunded product
df = df[~df["Invoice"].str.contains("C", na=False)]

print("Number of rows after subtracting refund data: ", df.shape[0])

Number of rows after subtracting refund data:  407695


# 3. Calculating RFM Metrics

One of the methods used in CRM Analytics is RFM Analysis. RFM is a combination of the initials of the terms Recency, Frequency and Monetary, each of which deals with a different characteristic of the customer. These characteristics are as follows;

* Recency is calculated by subtracting the date of the last interaction (shopping) of the customers from the date of the analysis (if it is a historical data set, the date of the analysis is determined close to the date of data collection), giving the date difference. The more recently a customer has shopped, the more likely they are to keep the brand (company) in mind for future purchases. It can also be used to identify customers who have not shopped for a long time and encourage them to visit the store again.

* Frequency indicates the number of times a customer makes a purchase. If it can be recognized that the customer's purchases are in a cycle, actions can be taken to predict when the customer will come back to the store or to remind them of their needs.

* The monetary value is the money left by the customer as a result of these purchases, indicating the total expenditure. While the monetary value can identify the customers who spend the most, it also carries the risk of alienating customers who spend relatively small amounts or new customers.

In [40]:
# Date Ranges in the Dataframe
print("Farthest date:", df["InvoiceDate"].min())
print("Nearest date:", df["InvoiceDate"].max())

# Since the Dataset is Old, We Need to Set the Date of Analysis as a More Closer Date
today_date = dt.datetime(2010, 12, 11)

Farthest date: 2009-12-01 07:45:00
Nearest date: 2010-12-09 20:01:00


In [41]:
# Calculating RFM Metrics and Assigning Them to a New Dataframe
# Group the data by customer ID and calculate the Recency, Frequency, and Monetary (RFM) metrics
rfm = df.groupby('Customer ID').agg({
                            'InvoiceDate': lambda InvoiceDate: (today_date - InvoiceDate.max()).days,  # Recency
                            'Invoice': lambda Invoice: Invoice.nunique(),  # Frequency
                            'TotalPrice': lambda TotalPrice: TotalPrice.sum()  # Monetary
})

# Rename the columns to recency, frequency, and monetary
rfm.columns = ["recency", "frequency", "monetary"]

# Display the first 10 rows of the new dataframe
rfm.head(10)

,recency,frequency,monetary
Customer ID,,,
12346.000,165,11,372.860
12347.000,3,2,1323.320
12348.000,74,1,222.160
12349.000,43,3,2671.140
12351.000,11,1,300.930
12352.000,11,2,343.800
12353.000,44,1,317.760
12355.000,203,1,488.210
12356.000,16,3,3562.250


In [42]:
# Describe of RFM Dataframe
rfm.describe().T

,count,mean,std,min,25%,50%,75%,max
recency,4314.000,91.270,96.944,1.000,18.000,53.000,136.000,374.000
frequency,4314.000,4.454,8.169,1.000,1.000,2.000,5.000,205.000
monetary,4314.000,2047.289,8912.523,0.000,307.950,705.550,1722.802,349164.350


In [43]:
#Monetary value 0 is not the result we want, we need to remove these records
rfm = rfm[rfm["monetary"] > 0]

# Describe of RFM Dataframe
rfm.describe().T

,count,mean,std,min,25%,50%,75%,max
recency,4312.000,91.173,96.861,1.000,18.000,53.000,136.000,374.000
frequency,4312.000,4.456,8.170,1.000,1.000,2.000,5.000,205.000
monetary,4312.000,2048.238,8914.481,2.950,307.988,706.020,1723.142,349164.350


# 4. Calculating RFM Scores

However, all of the values to be obtained as a result of these processes are relative values in terms of proximity & remoteness, frequency & infrequency and scarcity & abundance (according to whom according to what). Therefore, when conducting RFM Analysis, it would be useful to evaluate the data set within the data set itself and to bring these values to a more standardized interpretation by putting them into a rule. While doing this scoring, new values between 1 and 5 can be assigned to the Innovation, Frequency and Monetization values. The RFM value is formed by combining these values.

In [44]:
# Calculation of the Recency Score
rfm["recency_score"] = pd.qcut(rfm["recency"], 5, labels=[5, 4, 3, 2, 1])

#Calculation of the Frequency Score
#rank usage: it is a method for assigning a value to multiple repeated expressions (a value falling into 2 different categories, etc.).
rfm["frequency_score"] = pd.qcut(rfm['frequency'].rank(method="first"), 5, labels=[1, 2, 3, 4, 5])

#Calculating the Monetary Score
rfm["monetary_score"] = pd.qcut(rfm['monetary'], 5, labels=[1, 2, 3, 4, 5])

#Calculation of the RF Score
rfm["RF_SCORE"] = (rfm["recency_score"].astype(str) + rfm["frequency_score"].astype(str))

# View of Dataframe
rfm.head(10)

,recency,frequency,monetary,recency_score,frequency_score,monetary_score,RF_SCORE
Customer ID,,,,,,,
12346.000,165,11,372.860,2,5,2,25
12347.000,3,2,1323.320,5,2,4,52
12348.000,74,1,222.160,2,1,1,21
12349.000,43,3,2671.140,3,3,5,33
12351.000,11,1,300.930,5,1,2,51
12352.000,11,2,343.800,5,2,2,52
12353.000,44,1,317.760,3,1,2,31
12355.000,203,1,488.210,1,1,2,11
12356.000,16,3,3562.250,4,3,5,43


![RFM Segments](https://miro.medium.com/v2/resize:fit:720/format:webp/1*TjJt4rUiBtXLAF84--V-Cg.png)

* can't loose them → A class of customers who used to shop very frequently but have not shopped for a long time and should not be lost.
* at risk → A class of customers who shop relatively frequently but have not shopped for a long time.
hibernating → Customers who are literally asleep, both shopped infrequently and have not shopped for a long time.
* about to sleep → A class of customers who do not shop frequently and have not shopped for some time; they are heading towards sleep.
* need attention → This is the class of customers in the middle of the RF graph (33%), moving towards the risky group if not addressed.
* new customers → A class of customers who have not shopped frequently (maybe once) and have been shopping for a short period of time, they are considered as new customers.
* promising → New customers are the aged version of new customers :) they are customers who have not shopped frequently and have shopped for a short period of time.
* potential loyalists → Customers who shop moderately often and it has not been long since their last purchase.
* loyal customers → Customers who shop very often and it has been a short time since their last purchase.
* champions → They are our champions, our crown jewels! Customers who shop very often and have made their last purchase within a very short period of time.

In [45]:
rfm[rfm["RF_SCORE"] == "55"].head(10) #Champion Customers

,recency,frequency,monetary,recency_score,frequency_score,monetary_score,RF_SCORE
Customer ID,,,,,,,
12415.000,11,7,19543.840,5,5,5,55
12431.000,9,13,4370.520,5,5,5,55
12471.000,10,49,20139.740,5,5,5,55
12472.000,5,13,11308.480,5,5,5,55
12474.000,14,13,5048.660,5,5,5,55
12483.000,14,11,2635.600,5,5,5,55
12490.000,14,11,4405.710,5,5,5,55
12523.000,10,12,2582.380,5,5,5,55
12540.000,14,16,8854.250,5,5,5,55


In [46]:
# Converting RF_SCORE to Segment Name with Regex
seg_map = {
    r'[1-2][1-2]': 'hibernating',
    r'[1-2][3-4]': 'at_Risk',
    r'[1-2]5': 'cant_loose',
    r'3[1-2]': 'about_to_sleep',
    r'33': 'need_attention',
    r'[3-4][4-5]': 'loyal_customers',
    r'41': 'promising',
    r'51': 'new_customers',
    r'[4-5][2-3]': 'potential_loyalists',
    r'5[4-5]': 'champions'
}

rfm['segment'] = rfm['RF_SCORE'].replace(seg_map, regex=True)

In [47]:
# Segments' Statistical Info
rfm[["segment", "recency", "frequency", "monetary"]].groupby("segment").agg(["mean", "count"])

recency       frequency       monetary      
                       mean count      mean count     mean count
segment                                                         
about_to_sleep       53.819   343     1.201   343  441.320   343
at_Risk             152.159   611     3.074   611 1188.878   611
cant_loose          124.117    77     9.117    77 4099.450    77
champions             7.119   663    12.554   663 6852.264   663
hibernating         213.886  1015     1.126  1015  403.978  1015
loyal_customers      36.287   742     6.830   742 2746.067   742
need_attention       53.266   207     2.449   207 1060.357   207
new_customers         8.580    50     1.000    50  386.199    50
potential_loyalists  18.793   517     2.017   517  729.511   517
promising            25.747    87     1.000    87  367.087    87

# 5. Action Plans

* For example, we want to make a campaign specific to a segment: "cant loose them": For this we will select cant loose them customers.

In [48]:
new_df = pd.DataFrame()

new_df["new_customer_id"] = rfm[rfm["segment"] == "cant_loose"].index

new_df["new_customer_id"] = new_df["new_customer_id"].astype(int)

new_df.shape

(77, 1)

# THANK YOU!

* Thank you for reviewing this notebook up to here. If you like it and have any feedback we can meet in Upvotes and Comments :)